In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q "/content/drive/My Drive/color.zip" -d "/content/drive/My Drive/color_new"


replace /content/drive/My Drive/color_new/color/.idea/color.iml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras import regularizers
from keras.applications.inception_v3 import preprocess_input
from keras.layers.normalization import BatchNormalization
from keras.callbacks import CSVLogger, ReduceLROnPlateau, EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
batch_size = 32
epochs = 20
width=224
height=224
depth=3

In [ ]:

image_size = 0
default_image_size = tuple((224, 224))
directory_root = '/content/drive/My Drive/color_new/color'
INIT_LR = 1e-3

In [ ]:
def convert_image_to_array (image_dir):
  try:
    image = cv2.imread (image_dir)
    if image is not None:
      image = cv2.resize (image, default_image_size)
      return img_to_array(image)
    else:
      return np.array([])
  except Exception as e:
    print ("Exception:", e)
    return None

In [ ]:
image_ls =[]
label_ls = []

try:
  print ("Loading images")
  root = listdir (directory_root)
  for folder in root:
    if folder==".idea":
      root.remove(folder)
  for plant_disease_folder in root:
    print (f"[INFO] Processing {plant_disease_folder} ...")
    plant_disease_image_list = listdir(f"{directory_root}/{plant_disease_folder}/")
    #print (plant_disease_image_list)
    image_list = listdir (f"{directory_root}/{plant_disease_folder}/")
    for single_image in image_list [:600]:
      image_dir = f"{directory_root}/{plant_disease_folder}/{single_image}"
      if image_dir.endswith(".jpg") == True or image_dir.endswith(".JPG") == True:
        image_ls.append(convert_image_to_array(image_dir))
        label_ls.append(plant_disease_folder)
  print("[INFO] Image loading completed") 
except Exception as e:
    print(f"Error : {e}")

Loading images
[INFO] Processing Pepper,_bell___Bacterial_spot ...
[INFO] Processing Pepper,_bell___healthy ...


KeyboardInterrupt: ignored

In [ ]:
img_size = len (image_ls)

In [ ]:
img_list = []
label_list=[]
try:
  print ("Loading image")
  root = listdir (directory_root)
  for folder in root:
    if folder==".idea":
      root.remove(folder)
  for plant_disease_folder in root[10:11]:
    print (f"[INFO] Processing {plant_disease_folder} ...")
    plant_disease_image_list = listdir(f"{directory_root}/{plant_disease_folder}/")
    #print (plant_disease_image_list)
    image_list = listdir (f"{directory_root}/{plant_disease_folder}/")
    for single_image in image_list [1:2]:
      image_dir = f"{directory_root}/{plant_disease_folder}/{single_image}"
      if image_dir.endswith(".jpg") == True or image_dir.endswith(".JPG") == True:
        img_list.append(convert_image_to_array(image_dir))
        label_list.append(plant_disease_folder)
  print("[INFO] Image loading completed") 
except Exception as e:
    print(f"Error : {e}")


Loading image
[INFO] Processing Tomato___Septoria_leaf_spot ...
[INFO] Image loading completed


In [ ]:
#Transform image labels 

labelbinarizer = LabelBinarizer()
img_labels = labelbinarizer.fit_transform (label_ls)
pickle.dump (labelbinarizer, open('LabelTransform.pkl', 'wb'))
no_of_classes = len(labelbinarizer.classes_)
print (no_of_classes)

NameError: ignored

In [ ]:
print (labelbinarizer.classes_)

['Pepper,_bell___Bacterial_spot' 'Pepper,_bell___healthy'
 'Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy'
 'Tomato___Bacterial_spot' 'Tomato___Early_blight' 'Tomato___Late_blight'
 'Tomato___Leaf_Mold' 'Tomato___Septoria_leaf_spot'
 'Tomato___Spider_mites Two-spotted_spider_mite' 'Tomato___Target_Spot'
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus' 'Tomato___Tomato_mosaic_virus'
 'Tomato___healthy']


In [ ]:
np_img_list = np.array (image_ls, dtype = np.float16) / 225.0

In [ ]:
np_img_ls = np.array(img_list,dtype=np.float16) / 225.0

In [ ]:
#Splitting data into training and validation

x_train, x_val, y_train, y_val = train_test_split (np_img_list, img_labels, test_size = 0.2, random_state=42)

In [ ]:
aug = ImageDataGenerator (rotation_range = 25, width_shift_range = 0.1, height_shift_range = 0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest" )

In [ ]:
no_of_classes = 15

model = Sequential()
input_shape = (height,width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    input_shape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(64, (3, 3), padding="same",input_shape=input_shape))
model.add(Activation("relu"))
#model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
#model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding="valid"))
model.add(Activation("relu"))
#model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="valid"))
model.add(Activation("relu"))
#model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), activation='relu', padding='valid'))
#model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), activation='relu', padding='valid'))
#model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(512, (3, 3), activation='relu', padding='valid'))
#model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.6))
model.add(Conv2D(512, (3, 3), activation='relu', padding='valid'))
#.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(512, (3, 3), activation='relu', padding='valid'))
#model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.6))
model.add(Conv2D(512, (3, 3), activation='relu', padding='valid'))
#model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
#model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
#model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='relu'))
#model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.5))
model.add(Dense(no_of_classes))
model.add(Activation("softmax"))

In [ ]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 126, 126, 128)    

In [ ]:
model.load_weights ('/content/drive/My Drive/MyCNN/epochs:001-val_accuracy:0.079.hdf5')

In [ ]:
optimizer = Adam (lr=INIT_LR, decay = INIT_LR/epochs)
BS = 128
model.compile (loss="categorical_crossentropy", optimizer = optimizer, metrics = ['accuracy'])
print ("Train...")


Train...


In [ ]:
from keras.callbacks import *

filepath = "/content/drive/My Drive/MyCNN/epochs:{epoch:03d}-val_accuracy:{val_accuracy:.3f}.hdf5"

checkpoint = ModelCheckpoint (filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_val, y_val),
    initial_epoch=7,
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs, verbose=1,
    callbacks = callbacks_list
    )

Epoch 8/20
208/208 [==============================] - 5285s 25s/step - loss: 2.6830 - accuracy: 0.0665 - val_loss: 2.6773 - val_accuracy: 0.0781

Epoch 00008: val_accuracy improved from -inf to 0.07808, saving model to /content/drive/My Drive/MyCNN/epochs:008-val_accuracy:0.078.hdf5
Epoch 9/20
208/208 [==============================] - 5317s 26s/step - loss: 2.6801 - accuracy: 0.0708 - val_loss: 2.6775 - val_accuracy: 0.0787

Epoch 00009: val_accuracy improved from 0.07808 to 0.07868, saving model to /content/drive/My Drive/MyCNN/epochs:009-val_accuracy:0.079.hdf5
Epoch 10/20
208/208 [==============================] - 5335s 26s/step - loss: 2.6800 - accuracy: 0.0708 - val_loss: 2.6778 - val_accuracy: 0.0631

Epoch 00010: val_accuracy did not improve from 0.07868
Epoch 11/20
208/208 [==============================] - 5342s 26s/step - loss: 2.6812 - accuracy: 0.0690 - val_loss: 2.6774 - val_accuracy: 0.0673

Epoch 00011: val_accuracy did not improve from 0.07868
Epoch 12/20
208/208 [====

In [ ]:
plot_training(history)


In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_val, y_val)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('vgg_model_15_20_epoch.pkl', 'wb'))

In [ ]:
accu = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accu) + 1)
#Train and validation accuracy
plt.plot(epochs, accu, 'b', label='Training accurary')
plt.plot(epochs, val_acc, 'r', label='Validation accurary')
plt.title('Training and Validation accurary')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
model.predict(np_img_ls)

array([[0.068073  , 0.06854592, 0.0775432 , 0.072152  , 0.02383664,
        0.06484511, 0.07042155, 0.06890409, 0.07088616, 0.07664508,
        0.07566702, 0.07593176, 0.07378264, 0.04219518, 0.07057067]],
      dtype=float32)

In [ ]:
import tensorflow as tf

loaded_model = tf.keras.models.load_model('/content/drive/My Drive/MyCNN/epochs:001-val_accuracy:0.079.hdf5')
print (loaded_model.layers[0].input_shape)

(None, 256, 256, 3)


In [ ]:
result = loaded_model.predict_classes(np_img_ls)
print (result)

[2]


In [ ]:
print (result)

[2]


In [ ]:
print (get_label_name(result[0]))

NameError: ignored

In [ ]:
directory_root = '/content/drive/My Drive/PlantVillage/output'


In [ ]:

import time
import os

import numpy as np
import torch
import torch.optim as optim
from torch import nn
from torch.autograd import Variable

import torchvision
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
import torchvision.transforms as transforms
from torchvision import datasets

from itertools import accumulate
from functools import reduce


In [ ]:
model_urls = {
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth'   
}

In [ ]:
model_names = model_urls.keys()

input_sizes = {
    'vgg' : (224,224)
}

In [ ]:
models_to_test = [ 'vgg13']

batch_size = 20

In [ ]:
def diff_states(dict_canonical, dict_subset):
    names1, names2 = (list(dict_canonical.keys()), list(dict_subset.keys()))
    
    #Sanity check that param names overlap
    #Note that params are not necessarily in the same order
    #for every pretrained model
    not_in_1 = [n for n in names1 if n not in names2]
    not_in_2 = [n for n in names2 if n not in names1]
    assert len(not_in_1) == 0
    assert len(not_in_2) == 0

    for name, v1 in dict_canonical.items():
        v2 = dict_subset[name]
        assert hasattr(v2, 'size')
        if v1.size() != v2.size():
            yield (name, v1)    

In [ ]:
def load_defined_model(name, num_classes):
    
    model = models.__dict__[name](num_classes=num_classes)
    pretrained_state = model_zoo.load_url(model_urls[name])
    diff = [s for s in diff_states(model.state_dict(), pretrained_state)]
    print("Replacing the following state from initialized", name, ":",[d[0] for d in diff])
    
    for name, value in diff:
        pretrained_state[name] = value
    
    assert len([s for s in diff_states(model.state_dict(), pretrained_state)]) == 0
    
    model.load_state_dict(pretrained_state)
    return model, diff


In [ ]:
def filtered_params(net, param_list=None):
    def in_param_list(s):
        for p in param_list:
            if s.endswith(p):
                return True
        return False    
    #Caution: DataParallel prefixes '.module' to every parameter name
    params = net.named_parameters() if param_list is None else (p for p in net.named_parameters() if in_param_list(p[0]))
    return params

In [ ]:
def load_data(resize):

    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomSizedCrop(max(resize)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            #Higher scale-up for inception
            transforms.Scale(int(max(resize)/224*256)),
            transforms.CenterCrop(max(resize)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    data_dir = '/content/drive/My Drive/PlantVillage/output'
    dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
             for x in ['train', 'val']}
    dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                                   shuffle=True)
                    for x in ['train', 'val']}
    dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
    dset_classes = dsets['train'].classes
    
    return dset_loaders['train'], dset_loaders['val']

In [ ]:
def train(net, trainloader, param_list=None, epochs=15):
    def in_param_list(s):
        for p in param_list:
            if s.endswith(p):
                return True
        return False
    
    criterion = nn.CrossEntropyLoss()
    
    
    params = (p for p in filtered_params(net, param_list))
    
    #if finetuning model, turn off grad for other params
    if param_list:
        for p_fixed in (p for p in net.named_parameters() if not in_param_list(p[0])):
            p_fixed[1].requires_grad = False            
    
    #Optimizer as in paper
    optimizer = optim.SGD((p[1] for p in params), lr=0.001, momentum=0.9)

    losses = []
    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            
            loss = None
            # for nets that have multiple outputs such as inception
            if isinstance(outputs, tuple):
                loss = sum((criterion(o,labels) for o in outputs))
            else:
                loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.data
            if i % 30 == 29:
                avg_loss = running_loss / 30
                losses.append(avg_loss)
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, avg_loss))
                running_loss = 0.0

    print('Finished Training')
    return losses

#Get stats for training and evaluation in a structured way
#If param_list is None all relevant parameters are tuned,
#otherwise, only parameters that have been constructed for custom
#num_classes
def train_stats(m, trainloader, param_list = None):
    stats = {}
    params = filtered_params(m, param_list)    
    counts = 0,0
    for counts in enumerate(accumulate((reduce(lambda d1,d2: d1*d2, p[1].size()) for p in params)) ):
        pass
    stats['variables_optimized'] = counts[0] + 1
    stats['params_optimized'] = counts[1]
    
    before = time.time()
    losses = train(m, trainloader, param_list=param_list)
    stats['training_time'] = time.time() - before

    stats['training_loss'] = losses[-1] if len(losses) else float('nan')
    stats['training_losses'] = losses
    
    return stats

def evaluate_stats(net, testloader):
    stats = {}
    correct = 0
    total = 0
    
    before = time.time()
    for i, data in enumerate(testloader, 0):
        images, labels = data

        images, labels = (images.cuda()), (labels.cuda(async=True))

        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    accuracy = correct / total
    stats['accuracy'] = accuracy
    stats['eval_time'] = time.time() - before
    
    print('Accuracy on test images: %f' % accuracy)
    return stats


def train_eval(net, trainloader, testloader, param_list=None):
    print("Training..." if not param_list else "Retraining...")
    stats_train = train_stats(net, trainloader, param_list=param_list)
    
    print("Evaluating...")
    net = net.eval()
    stats_eval = evaluate_stats(net, testloader)
    
    return {**stats_train, **stats_eval}

In [ ]:
stats = []
num_classes = 15
print("RETRAINING")

for name in models_to_test:
    print("")
    print("Targeting %s with %d classes" % (name, num_classes))
    print("------------------------------------------")
    model_pretrained, diff = load_defined_model(name, num_classes)
    final_params = [d[0] for d in diff]
    #final_params = None
    
    resize = [s[1] for s in input_sizes.items() if s[0] in name][0]
    print("Resizing input images to max of", resize)
    trainloader, testloader = load_data(resize)
    
        
    pretrained_stats = train_eval(model_pretrained, trainloader, testloader, final_params)
    pretrained_stats['name'] = name
    pretrained_stats['retrained'] = True
    pretrained_stats['shallow_retrain'] = True
    stats.append(pretrained_stats)
    
    print("")

RETRAINING

Targeting vgg13 with 15 classes
------------------------------------------
Replacing the following state from initialized vgg13 : ['classifier.6.weight', 'classifier.6.bias']
Resizing input images to max of (224, 224)


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:698: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Retraining...
[1,    30] loss: 2.400
[1,    60] loss: 1.769
[1,    90] loss: 1.373
[1,   120] loss: 1.339
[1,   150] loss: 1.329
[1,   180] loss: 1.180
[1,   210] loss: 1.222
[1,   240] loss: 1.190
[1,   270] loss: 1.106
[1,   300] loss: 1.075
[1,   330] loss: 1.015
[1,   360] loss: 0.979
[1,   390] loss: 0.968
[1,   420] loss: 1.047
[1,   450] loss: 0.949
[1,   480] loss: 0.990
[1,   510] loss: 0.941
[1,   540] loss: 1.020
[1,   570] loss: 0.994
[1,   600] loss: 1.037
[1,   630] loss: 0.929
[1,   660] loss: 0.931
[1,   690] loss: 0.907
[1,   720] loss: 0.940
[1,   750] loss: 1.027
[1,   780] loss: 0.795
[1,   810] loss: 0.997
[1,   840] loss: 0.899
[1,   870] loss: 0.887
[1,   900] loss: 0.824
[2,    30] loss: 0.820
[2,    60] loss: 0.854
[2,    90] loss: 0.786
[2,   120] loss: 0.913
[2,   150] loss: 0.870
[2,   180] loss: 0.794
[2,   210] loss: 0.804
[2,   240] loss: 0.883
[2,   270] loss: 0.949
[2,   300] loss: 0.845
[2,   330] loss: 0.769
[2,   360] loss: 0.872
[2,   390] loss: 0.8